In [0]:
import numpy as np
from model.nn import *
import torch
import mlflow
import mlflow.pytorch
from components.helper import *
%load_ext autoreload
%autoreload 2

In [0]:
import pickle
import numpy as np

# Define DBFS and local paths
dbfs_file_path = "dbfs:/mnt/proddatalake/dev/TierImminent/intermediate_data/borgata_new_02102025_np_dict.pkl"
local_file_path = "/Workspace/Users/609399@mgmresorts.com/temp.pkl"

# Copy the file from DBFS to local storage
dbutils.fs.cp(dbfs_file_path, f"file:{local_file_path}")

# Load the NumPy array from the pickle file
with open(local_file_path, "rb") as f:
    loaded_array = pickle.load(f)

In [0]:
y = loaded_array['label_tensor']
X = loaded_array['train_tensor']
length = loaded_array['length_tensor']
ids = loaded_array['id_tensor']

In [0]:
sum(y)

In [0]:
train_labels, train_outputs, labels, outputs, run_id = train_lstm(X, y, length, ids, input_dim=X.shape[2], hidden_dim=64, num_layers=1, batch_size=16, learning_rate=0.001, num_epochs=10, balance = 40.0, logging=True)

In [0]:
mlflow.set_registry_uri("databricks-uc")
model_name = "data_science_mart.tierimminent_cleaned.borgata_lstm_model_new"
mlflow.register_model(f"runs:/{run_id}/lstm_pytorch_model_{date.today().strftime('%Y-%m-%d')}", model_name)

In [0]:
outputs = torch.tensor(outputs, dtype=torch.float32).cpu()
outputs = torch.sigmoid(outputs)

train_outputs = torch.tensor(outputs, dtype=torch.float32).cpu()
train_outputs = torch.sigmoid(train_outputs)

fpr, tpr, thresholds = roc_curve(labels, outputs)
auc_score = roc_auc_score(labels, outputs)
plt.figure()
plt.plot(fpr, tpr, label=f"ROC Curve (AUC = {auc_score:.2f})")
plt.plot([0, 1], [0, 1], 'k--', label="Random Guess")
plt.xlabel("False Positive Rate")
plt.ylabel("True Positive Rate")
plt.title("ROC Curve")
plt.legend(loc="best")
plt.grid()
for i in range(0, len(thresholds), len(thresholds)//20):
    plt.text(fpr[i], tpr[i], f'{thresholds[i]:.2f}', fontsize=8, color='red')
#plt.savefig("/Workspace/Users/609399@mgmresorts.com/Tier Imminent/model_performance_metric/sim_2024_roc.png")
plt.show()
plt.close() 